### Example for predicting using the RNN

This is an example for running the RNN and get water demand predictions.

Requirements:
- Python 3.6+
- NumPy 1.14+ (http://www.numpy.org/)

To run this Notebook locally:
- Jupyter (https://jupyter.org/)

___Note:___ This model is an online prediction model. The current version of code does provides a separate option for training the RNN. Refer to [training_example.ipynb](./training_example.ipynb) for training the RNN.

#### Step 1: Import external libraries and the `rnn` package

In [1]:
import os
from rnn import *

#### Step 2: Specify the location on disk of the data directory and the input CSV file (optional)

In [2]:
DATA_LOCATION = os.getcwd()
INPUT_FILE = os.path.join(DATA_LOCATION, 'input_files', 'water_6_17_reduced_2_cap_rs.csv')

#### Step 3: To initialize the predictor RNN, create a `rnn_predict` object.

Required parameters:
- `weights`: Location on disk of the weights file. Typically a Python serialized object (Pickle) file.

Optional parameters:
- `alpha`: Learning rate for the network, typically a value in the range `(0, 1)`. Default value: `0.38`.
- `log`: Boolean value specifying if the execution should be logged to the console. Recommended if running from the command line or in an IDE. Default value: `False`.
- `h_bias`: Boolean value specifying if hidden layer should have a bias node. Currently trained models include a hidden layer bias. Default value: `True`.

In [3]:
predictor = rnn_predict(os.path.join(DATA_LOCATION, 'input_files', 'daily_rnn_retrained_16.pickle'))

At this point, the model is ready to make predictions.

#### Step 4a: Give the network a list of inputs to make predictions

Required parameters:
- `inputs`: A list of `[day_of_the_year, maximum_temperature, precipitation]` lists. These should be real world observed or predicted values.

Optional parameters:
- `networks`: A list of the same length as `inputs` list with values of the range `[0, 11]`, specifying what network each prediction to run on, typically `num_month - 1`. If not specified, all predictions will be made for the month of January.
- `targets`: A list of the same length as `inputs` of the observed values for the day. The target values are required for updating the weights of the model.
- `recal`: Boolean value specifying if the network weights should be recalculated. Recommended if using observed demand as target, to improve future predictions. If set to `True`, requires `targets` to be specified. Default value: `False`.

Output:
- `predictions`: A list of the same length as `inputs` of the predictions made by the network.

In [4]:
inputs = [[1, 71, 0], [2, 101, 121]]
networks = [0, 0]

predictions = predictor.predict(inputs, networks=networks)

The prediction for inputs  [1, 71, 0]  for network  0  is 110.72
The prediction for inputs  [2, 101, 121]  for network  0  is 110.71


#### Step 4b: Or give the network a list of inputs to make predictions for and targets to continue learning from

___Note:___ Once retrained, the weights are updated in the object immediately and cannot be undone, unless restored from a previously stored weights file.

In [5]:
inputs = [[1, 71, 0], [2, 101, 121]]
networks = [0, 0]
targets = [106.53, 110.72]

predictions = predictor.predict(inputs, networks=networks, targets=targets, recal=True)

The target for inputs  [1, 71, 0]  for network  0  was  106.53 and the prediction was 110.72, error observed was 3.93%. 
The target for inputs  [2, 101, 121]  for network  0  was  110.72 and the prediction was 115.86, error observed was 4.64%. 


The network is now retrained and has updated the weights.

(Observe the difference in the second prediction because the weights were updated after predicting the first value.)

#### Step 4c: Or give the network a CSV file of inputs, targets and networks to make predictions for and to continue learning from

___Note:___ Once retrained, the weights are updated in the object immediately and cannot be undone, unless restored from a previously stored weights file.

In [6]:
inputs, networks, targets = read_input_file(INPUT_FILE)

predictions = predictor.predict(inputs[:5], networks=networks[:5], targets=targets[:5], recal=True)

The target for inputs  [1, 71, 0]  for network  0  was  106.53 and the prediction was 115.19, error observed was 8.13%. 
The target for inputs  [2, 101, 121]  for network  0  was  114.81 and the prediction was 114.44, error observed was 0.32%. 
The target for inputs  [3, 107, 116]  for network  0  was  111.63 and the prediction was 113.52, error observed was 1.70%. 
The target for inputs  [4, 24, 81]  for network  0  was  111.85 and the prediction was 113.78, error observed was 1.72%. 
The target for inputs  [5, -69, 39]  for network  0  was  111.87 and the prediction was 113.83, error observed was 1.75%. 


#### Step 5: Save the predictions to a CSV file

Required parameters:
- `targets`: As above.
- `predictions`: As above.
- `output_file`: Location on disk of the output file, with the name.

In [7]:
write_output_to_file(targets, predictions, os.path.join(DATA_LOCATION, 'predictions_2017.csv'))

#### Step 6: Store the weights for future use

Store the network state for future re-use.

In [8]:
predictor.save(os.path.join(DATA_LOCATION, 'daily_rnn_retrained_17.pickle'))

Further documentation about other functions implemented is available within the source files. The shared variables and parameters are documented in `__init__.py`.